In [3]:
#we assume all the images have same resolution
import os
import json
import numpy as np
import shutil
import random
import cv2 as cv

labels_in       = 'D:\SIGNATE\Signate_3rd_AI_edge_competition\data_for_yolo_training.txt'
labels_out      = 'D:\SIGNATE\Signate_3rd_AI_edge_competition\data_for_yolo_training_inpaint3.txt'
out_dir         = 'D:\SIGNATE\Signate_3rd_AI_edge_competition\imgs/inpaint/' #the dir will be created
generate_images = 10000
out_file        = open(labels_out, "w")
max_x           = 1935
max_y           = 949

os.makedirs(out_dir, exist_ok=True)



def insert_img(img_out, img_in, box):
    
    cx  = (box[0]+box[2])/2.0
    cy  = (box[1]+box[3])/2.0
    w = (box[2]-box[0])/1.4 #1.4 is used to adjust progress of weights
    h = (box[3]-box[1])/1.4
    for x in range(box[0], box[2]):
        wx = 1.0 - pow(min(np.abs(cx-x)/w, 1), 3.0)
        for y in range(box[1], box[3]):
            wy = 1.0-pow(min(np.abs(cy-y)/h, 1), 3.0)
            wxy = wx*wy
            img_out[y, x, ...] = np.clip(wxy*img_in[y, x, ...] + (1.0-wxy)*img_out[y, x, ...], 0, 254)
    


def is_intersect(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    if interArea > 0: return True
    return False
    

def is_intersect_multi(boxA, boxes):
    for i in range (0, len(boxes)):
        if is_intersect(boxA, boxes[i]):
            return True
    return False



with open(labels_in) as f:  lines    = f.readlines()
    
for i in range (0, len(lines)): lines[i] = lines[i].split()
f.close()

tot_imgs1 = len(lines)-1

print("len(lines) = ", len(lines))
print("tot_imgs1 = ", tot_imgs1)

gen = 0
while gen<generate_images:    
    index1 = random.randint(0,tot_imgs1)
    index2 = random.randint(0,tot_imgs1)
        
    #print("index1 = {}, index2 = {}".format(index1, index2) )
    #print("\rlines[{}][0] = {},  lines[{}][0] = {}".format(index1, lines[index1][0], index2, lines[index2][0]), end="" )
    im1 = cv.imread(lines[index1][0])
    im2 = cv.imread(lines[index2][0])
    
    
    boxes1 = np.array([np.array(list(map(int, box.split(',')))) for box in lines[index1][1:]])
    boxes2 = np.array([np.array(list(map(int, box.split(',')))) for box in lines[index2][1:]])
    
    added = 0
    str_to_write = out_dir+str(gen)+'.jpg'
    for i in range (0, len(boxes2)):
        box = boxes2[i]
        if box[4] > 1:
            continue
        if (box[2]-box[0]) * (box[3]-box[1]) < 2000: # we do not want small boxes
            continue
        if (box[2]-box[0]) * (box[3]-box[1]) > 60000: #we do not want too big boxes
            continue    
        if is_intersect_multi(box, boxes1):
            continue
        added += 1
        over = random.randint(0,25)
        box_to_inpaint = np.asarray( (max(box[0]-over,0), max(box[1]-over,0), min(box[2]+over,max_x), min(box[3]+over,max_y)) )
        insert_img(im1, im2, box_to_inpaint)
        
        
        str_to_write += ' '+str(box[0])+','+str(box[1])+','+str(box[2])+','+str(box[3])+','+str(box[4])
    
    for i in range (0, len(boxes1)):
        box = boxes1[i]
        if box[4] > 1:
            continue
        if (box[2]-box[0]) * (box[3]-box[1]) < 300: #we do not want small boxes
            continue
        str_to_write += ' '+str(box[0])+','+str(box[1])+','+str(box[2])+','+str(box[3])+','+str(box[4])
    if added<3:
        continue
    if added>10:
        continue
    
    if str_to_write != out_dir+str(gen)+'.jpg':
        out_file.write(str_to_write+'\n')
        
    #if gen % 200 ==1: print(gen)
        
    print("\r{} :".format(gen) + out_dir+str(gen)+'.jpg                ', end="" )
    cv.imwrite(out_dir+str(gen)+'.jpg', im1)
    
    gen+=1
out_file.close()   

len(lines) =  14382
tot_imgs1 =  14381
9999 :D:\SIGNATE\Signate_3rd_AI_edge_competition\imgs/inpaint/9999.jpg                

In [4]:
#we assume all the images have same resolution
#inpaint with crops
import os
import json
import numpy as np
import shutil
import random
import cv2 as cv

labels_in       = 'D:\SIGNATE\Signate_3rd_AI_edge_competition\data_for_yolo_training.txt'
labels_out      = 'D:\SIGNATE\Signate_3rd_AI_edge_competition\data_for_yolo_training_inpaint_crop.txt'
out_dir         = 'D:\SIGNATE\Signate_3rd_AI_edge_competition\imgs/inpaint_crop/'  #the dir will be created
generate_images = 10000
out_file        = open(labels_out, "w")
max_x           = 1935
max_y           = 949

os.makedirs(out_dir, exist_ok=True)



def insert_img(img_out, img_in, box):
    img_out[box[1]:box[3], box[0]:box[2], ...] = img_in[box[1]:box[3], box[0]:box[2], ...]


def is_intersect(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    if interArea > 0: return True
    return False
    

def is_intersect_multi(boxA, boxes):
    for i in range (0, len(boxes)):
        if is_intersect(boxA, boxes[i]):
            return True
    return False



with open(labels_in) as f:  lines    = f.readlines()
for i in range (0, len(lines)): lines[i] = lines[i].split()
f.close()

tot_imgs1 = len(lines)-1

gen = 0
while gen<generate_images:    
    index1 = random.randint(0,tot_imgs1)
    index2 = random.randint(0,tot_imgs1)
        
        
    im1 = cv.imread(lines[index1][0])
    im2 = cv.imread(lines[index2][0])
    
    
    
    boxes1 = np.array([np.array(list(map(int, box.split(',')))) for box in lines[index1][1:]])
    boxes2 = np.array([np.array(list(map(int, box.split(',')))) for box in lines[index2][1:]])
    
    added = 0
    str_to_write = out_dir+str(gen)+'.jpg'
    for i in range (0, len(boxes2)):
        box = boxes2[i]
        if box[4] > 1:
            continue
        if (box[2]-box[0]) * (box[3]-box[1]) < 2000: #too small
            continue
        if (box[2]-box[0]) * (box[3]-box[1]) > 60000: #too big
            continue    
            
        #crop it    
        height_box = box[3]-box[1]
        if box[4]==0:#car
            new_height_box = height_box/100.0*random.randint(60,90)
        else:#pedestrian, we crop it more
            new_height_box = height_box/100.0*random.randint(30,90)
        box[3] = int(box[1]+new_height_box)
        
            
            
        if is_intersect_multi(box, boxes1):
            continue
        added += 1
        over = random.randint(0,25)
        box_to_inpaint = np.asarray( (max(box[0]-over,0), max(box[1]-over,0), min(box[2]+over,max_x), box[3]) )
        insert_img(im1, im2, box_to_inpaint)
        
        
        str_to_write += ' '+str(box[0])+','+str(box[1])+','+str(box[2])+','+str(box[3])+','+str(box[4])
        #cv.rectangle(im1, (box[0],box[1]), (box[2],box[3]), (0,0,255), 3, 1)
    
    for i in range (0, len(boxes1)):
        box = boxes1[i]
        if box[4] > 1:
            continue
        if (box[2]-box[0]) * (box[3]-box[1]) < 300: #we do not want small boxes
            continue
        str_to_write += ' '+str(box[0])+','+str(box[1])+','+str(box[2])+','+str(box[3])+','+str(box[4])
        
    if added<3:
        continue
    if added>10:
        continue
    
    if str_to_write != out_dir+str(gen)+'.jpg':
        out_file.write(str_to_write+'\n')
        
    #if gen % 200 ==1: print(gen)
        
    print("\r{} :".format(gen) + out_dir + str(gen) +'.jpg                ', end="" )
    cv.imwrite(out_dir+str(gen)+'.jpg', im1)
    
    gen+=1
out_file.close()   

0 :D:\SIGNATE\Signate_3rd_AI_edge_competition\imgs/inpaint_crop/0.jpg                1
200 :D:\SIGNATE\Signate_3rd_AI_edge_competition\imgs/inpaint_crop/200.jpg                201
400 :D:\SIGNATE\Signate_3rd_AI_edge_competition\imgs/inpaint_crop/400.jpg                401
600 :D:\SIGNATE\Signate_3rd_AI_edge_competition\imgs/inpaint_crop/600.jpg                601
800 :D:\SIGNATE\Signate_3rd_AI_edge_competition\imgs/inpaint_crop/800.jpg                801
1000 :D:\SIGNATE\Signate_3rd_AI_edge_competition\imgs/inpaint_crop/1000.jpg                1001
1200 :D:\SIGNATE\Signate_3rd_AI_edge_competition\imgs/inpaint_crop/1200.jpg                1201
1400 :D:\SIGNATE\Signate_3rd_AI_edge_competition\imgs/inpaint_crop/1400.jpg                1401
1600 :D:\SIGNATE\Signate_3rd_AI_edge_competition\imgs/inpaint_crop/1600.jpg                1601
1800 :D:\SIGNATE\Signate_3rd_AI_edge_competition\imgs/inpaint_crop/1800.jpg                1801
2000 :D:\SIGNATE\Signate_3rd_AI_edge_competition\imgs/inpaint